In [3]:
import pandas as pd
import rdkit
from rdkit import Chem
import re
import numpy as np

In [38]:
ab_int = pd.read_csv(r'C:\NIRSII\To_git\data\Antibodies\final\Отправить\antibody_interactions.csv', index_col = 0)
ab_mol = pd.read_csv(r'C:\NIRSII\To_git\data\Antibodies\final\Отправить\molecules_from_antibodies.csv', index_col = 0)
ab_mol['content'] = ab_mol['content'].replace(r'[\t\n\r ]+', '', regex=True)
ab = pd.read_csv(r'C:\NIRSII\To_git\data\Antibodies\final\Отправить\antibodies.csv', index_col = 0)

apt_int = pd.read_csv(r'C:\NIRSII\To_git\merging\13_02_2025\aptamers_interactions.csv', index_col = 0)
apt = pd.read_csv(r'C:\NIRSII\To_git\merging\13_02_2025\apt_final.csv', index_col = 0)
apt_mol = pd.read_csv(r'C:\NIRSII\To_git\merging\13_02_2025\molecules_from_aptamers.csv', index_col = 0)

In [39]:
apt_int.rename(columns={'target_name': 'ligand_name'}, inplace=True)

In [40]:
ab_mol.head(2)

,name,type,content,representation_type
0,cep1 peptide (from enolase),peptide,ACACKIHACIREIFDSCIRGNPTVECK,sequence
1,glucose-6-phosphate isomerase,peptide,ACACFEQLLSGAHWMDQHFCA,sequence


In [41]:
def smiles_to_mol_to_smiles(smiles_string):
    """
    Converts a SMILES string to a Mol object, then back to a SMILES string.

    Args:
        smiles_string: The input SMILES string.

    Returns:
        A tuple containing:
            - The canonical SMILES string.
            - The Mol object.
        Returns None, None if the SMILES string is invalid.
    """
    # Convert SMILES to Mol object
    mol = Chem.MolFromSmiles(smiles_string)

    # Check if the Mol object is valid
    if mol is None:
        print("Invalid SMILES string")

    # Convert Mol object back to SMILES (canonical)
    canonical_smiles = Chem.MolToSmiles(mol, canonical=True, isomericSmiles=False)

    return canonical_smiles

In [42]:
# Перевод в каноничные smiles
n =0
for i in range(ab_mol.shape[0]):
    if ab_mol.loc[i, 'representation_type'] == 'smiles':
        smi = ab_mol.loc[i, 'content']
        print(smi)
        try:
            canonical_smiles = smiles_to_mol_to_smiles(smi)
            ab_mol.loc[i, 'content'] = canonical_smiles
            print(f'Успешно! {canonical_smiles}')
            n+=1
            continue  # Перейти к следующей итерации цикла, если успешно

        except Exception as e:
            print(f"Проблема со строкой {i}: {smi} {e},\tПопробуем добавить скобку")
            smi += ')'
            print(f'Строка со скобкой: {smi}')
            try:
                canonical_smiles = smiles_to_mol_to_smiles(smi)
                ab_mol.loc[i, 'content'] = canonical_smiles
                print(f'Теперь успешно! {canonical_smiles}')
                n+=1
            except Exception as e:
                print(f"Всё ещё проблема со строкой {i}: {smi} {e}\tПопробуем добавить другую скобку")
                smi = smi[:-1] + ']'
                print(f'Строка с новой скобкой: {smi}')
                try:
                    canonical_smiles = smiles_to_mol_to_smiles(smi)
                    ab_mol.loc[i, 'content'] = canonical_smiles
                    print(f'Вот теперь успешно! {canonical_smiles}')
                    n+=1
                except Exception as e:
                    print(f"До сих пор проблема со строкой {i}: {smi} {e}")
                    ab_mol.loc[i, 'content'] = smi[:-1]


C(CCNCC(CCN)O)CC(C(=O)O)
Успешно! NCCC(O)CNCCCCCC(=O)O
C(CNC(CO)(CO)CO)CNC(CO)(CO)
Успешно! OCC(CO)NCCCNC(CO)(CO)CO
C(COCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCO)O
Успешно! OCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCO
C(CS(=O)(=O)O)NC(CO)(CO)
Успешно! O=S(=O)(O)CCNC(CO)CO
C([C@@H]1[C@H]([C@@H]([C@@H]([C@H](O1)OC[C@@H]2[C@H]([C@@H]([C@@H]([C@H](O2)O)O)O)O)O)O)O)
Успешно! CC1OC(OCC2OC(O)C(O)C(O)C2O)C(O)C(O)C1O
C/C(=C\C=C\C1=[N+](C2=C(C1(C)C)C=C(C=C2)S(=O)(=O)O)CCS(=O)(=O)O)/C=C/C=C/3\C(C4=C(N3CCS(=O)(=O)O)C=CC(=C4)S(=O)(=O)O)(C)
Успешно! CC(C=CC=C1C(C)c2cc(S(=O)(=O)O)ccc2N1CCS(=O)(=O)O)=CC=CC1=[N+](CCS(=O)(=O)O)c2ccc(S(=O)(=O)O)cc2C1(C)C
C/C=C/O[C@@]1(C[C@H]([C@H]([C@H](O1)[C@@H](CO)OC)O)O)C(=O
Invalid SMILES string
Проблема со строкой 765: C/C=C/O[C@@]1(C[C@H]([C@H]([C@H](O1)[C@@H](CO)OC)O)O)C(=O Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmile

[02:06:55] SMILES Parse Error: extra open parentheses for input: 'C/C=C/O[C@@]1(C[C@H]([C@H]([C@H](O1)[C@@H](CO)OC)O)O)C(=O'
[02:06:55] SMILES Parse Error: extra open parentheses for input: 'C1=C(C=C(C(=C1I)OC2=CC(=C(C(=C2)I)O)I)I)C[C@@H](C(=O)O'
[02:06:55] SMILES Parse Error: syntax error while parsing: C1=C(C=C(C(=C1[N+](=O)[O-])O)I)CC(=O)NCCCCCC(=O)[O-
[02:06:55] SMILES Parse Error: Failed parsing SMILES 'C1=C(C=C(C(=C1[N+](=O)[O-])O)I)CC(=O)NCCCCCC(=O)[O-' for input: 'C1=C(C=C(C(=C1[N+](=O)[O-])O)I)CC(=O)NCCCCCC(=O)[O-'
[02:06:55] SMILES Parse Error: syntax error while parsing: C1=C(C=C(C(=C1[N+](=O)[O-])O)I)CC(=O)NCCCCCC(=O)[O-)
[02:06:55] SMILES Parse Error: Failed parsing SMILES 'C1=C(C=C(C(=C1[N+](=O)[O-])O)I)CC(=O)NCCCCCC(=O)[O-)' for input: 'C1=C(C=C(C(=C1[N+](=O)[O-])O)I)CC(=O)NCCCCCC(=O)[O-)'
[02:06:55] SMILES Parse Error: extra open parentheses for input: 'C1=CC(=C(C=C1CC(=O)O)[N+](=O)[O-]'
[02:06:55] SMILES Parse Error: extra open parentheses for input: 'C1=CC(=C(C=C1S(=O

In [43]:
ab_mol

,name,type,content,representation_type
0,cep1 peptide (from enolase),peptide,ACACKIHACIREIFDSCIRGNPTVECK,sequence
1,glucose-6-phosphate isomerase,peptide,ACACFEQLLSGAHWMDQHFCA,sequence
2,ring-infected erythrocyte surface antigen peptide,peptide,ACAEENVEENVEENVEENVGG,sequence
3,p/a#1 epitope peptide,peptide,ACEAPAPAAPA,sequence
4,hiv fusion peptide 512-519,peptide,ACEAVGLGAVF,sequence
...,...,...,...,...
3507,major prion protein,protein,VVGGLGGYMLGSAMSRPMIHFGNDWEDRYYRENMYRYPNQVYYRPV...,sequence
3508,coagulation factor xii,protein,VVGGLVALRGAHPYIAALYWGHSFCAGSLIAPCWVLTAAHCLQDRP...,sequence
3509,suppressor of tumorigenicity 14 protein,protein,VVGGTDADEGEWPWQVSLHALGQGHICGASLISPNWLVSAAHCYID...,sequence
3510,vascular endothelial growth factor,protein,VVKFMDVYQRSYCHPIETLVDIFQEYPDEIEYIFKPSCVPLMRCGG...,sequence


In [44]:
ab_mol['content'].value_counts()

content
C[CH]Cc1ccccc1                           2
CN1C2CCC1C(C(=O)O)C(OC(=O)c1ccccc1)C2    2
ACAEENVEENVEENVEENVGG                    1
ACEAPAPAAPA                              1
ACEAVGLGAVF                              1
                                        ..
ACEDAEFRHDSGYEVHHQKNH2                   1
ACEDVPDYAS                               1
ACEDVPDYASLNH2                           1
ACEDGLHDPRNH2                            1
ACACFEQLLSGAHWMDQHFCA                    1
Name: count, Length: 3510, dtype: int64

In [45]:
ab_int.merge(apt_int, on='ligand_name')

,antibody_name,ligand_name,kd,aptamer_name,pKd,measurement,affinity
0,8uf7_B_A,prothrombin,NaN,APIPred_5,NaN,NaN,NaN
1,8uf7_B_A,prothrombin,NaN,APIPred_471,NaN,NaN,NaN
2,8uf7_B_A,prothrombin,NaN,APIPred_533,NaN,NaN,NaN
3,8uf7_B_A,prothrombin,NaN,APIPred_629,NaN,NaN,NaN
4,8uf7_B_A,prothrombin,NaN,APIPred_868,NaN,NaN,NaN
5,8uf7_B_A,prothrombin,NaN,APIPred_941,NaN,NaN,NaN
6,8uf7_B_A,prothrombin,NaN,APIPred_992,NaN,NaN,NaN
7,8uf7_B_A,prothrombin,NaN,APIPred_1006,NaN,NaN,NaN
8,8uf7_B_A,prothrombin,NaN,APIPred_1178,NaN,NaN,NaN
9,8uf7_B_A,prothrombin,NaN,APIPred_1776,NaN,NaN,NaN


In [46]:
apt_mol_smi = apt_mol[apt_mol['representation_type'] == 'smiles']

In [47]:
apt_mol_smi.reset_index(inplace=True)

In [48]:
ab_mol_smi = ab_mol[ab_mol['representation_type'] == 'smiles']

In [49]:
ab_mol_smi.reset_index(inplace=True)

In [50]:
apt_mol_smi['content'].value_counts()

content
Nc1ncnc2c1ncn2C1OC(COP(=O)(O)OP(=O)(O)OP(=O)(O)O)C(O)C1O                      142
Cn1c(=O)c2[nH]cnc2n(C)c1=O                                                     41
NC(N)=NCCCC(N)C(=O)O                                                           33
NCC1OC(OC2C(N)CC(N)C(OC3OC(CNc4c5ccccc5nc5ccccc45)C(O)C(N)C3O)C2O)C(N)CC1O     28
Nc1ncnc2c1ncn2C1OC(CO)C(O)C1O                                                  26
                                                                             ... 
Clc1ccc(C(c2ccccc2Cl)C(Cl)(Cl)Cl)cc1                                            1
CN1CCC23c4c5ccc(O)c4OC2C(O)C=CC3C1C5                                            1
CC(C)(c1ccc(O)cc1)c1ccc(O)cc1                                                   1
CC(CCC(=O)O)C1CCC2C3C(O)CC4CC(O)CCC4(C)C3CC(O)C12C                              1
CN(C)C1C(=O)C(C(N)=O)=C(O)C2(O)C(=O)C3=C(O)c4c(O)cccc4C(C)(O)C3C(O)C12          1
Name: count, Length: 359, dtype: int64

In [59]:
apt_mol_smi_ab_mol_smi_merge_on_content = apt_mol_smi.merge(ab_mol_smi, on='content', how='inner')

In [58]:
apt_mol_ab_mol_merge_on_name = apt_mol.merge(ab_mol, on='name')

In [56]:
apt_mol_ab_mol_merge_on_content = apt_mol.merge(ab_mol, on='content')

In [60]:
apt_mol_ab_mol_merge_on_content.to_csv('apt_mol_ab_mol_merge_on_content.csv')
apt_mol_ab_mol_merge_on_name.to_csv('apt_mol_ab_mol_merge_on_name.csv')
apt_mol_smi_ab_mol_smi_merge_on_content.to_csv('apt_mol_smi_ab_mol_smi_merge_on_content.csv')